# Pluggable TTA Implementation example

## Import Libraries

In [ ]:
from os import path

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset

from torchvision import datasets, utils, transforms
from torchinfo import summary

from ttadapters.datasets import GOT10kDatasetForObjectTracking, PairedGOT10kDataset

import numpy as np
import pandas as pd

import wandb
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

datasets.utils.tqdm = tqdm

In [ ]:
PROJECT_NAME = "APT_PLUGIN"
RUN_NAME = "RT-DETR_R50_APT"

# WandB Initialization
#wandb.init(project=PROJECT_NAME, name=RUN_NAME)

### Check GPU Availability

In [ ]:
!nvidia-smi

In [ ]:
# Set CUDA Device Number 0~7
DEVICE_NUM = 0
ADDITIONAL_GPU = 0

if torch.cuda.is_available():
    if ADDITIONAL_GPU:
        torch.cuda.set_device(DEVICE_NUM)
        device = torch.device("cuda")
    else:
        device = torch.device(f"cuda:{DEVICE_NUM}")
else:
    device = torch.device("cpu")
    DEVICE_NUM = -1

print(f"INFO: Using device - {device}" + (f":{DEVICE_NUM}" if ADDITIONAL_GPU else ""))

## Load Dataset

### Original Dataset

In [ ]:
DATA_ROOT = path.join(".", "data")

train_dataset = GOT10kDatasetForObjectTracking(root=DATA_ROOT, force_download=False, train=True)
valid_dataset = GOT10kDatasetForObjectTracking(root=DATA_ROOT, force_download=False, valid=True)
test_dataset = GOT10kDatasetForObjectTracking(root=DATA_ROOT, force_download=False, train=False)

print(f"INFO: Dataset loaded successfully. Number of samples - Train({len(train_dataset)}), Valid({len(valid_dataset)}), Test({len(test_dataset)})")

In [ ]:
train_dataset.targets

In [ ]:
train_dataset[0]

In [ ]:
train_dataset[1]

In [ ]:
test_dataset[-1]

### Paired Dataset

In [ ]:
# Define image size for resizing
ORIGINAL_SIZE = train_dataset[0][0].size
IMG_SIZE = 800

# Define image normalization parameters
IMG_NORM = dict(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

print("INFO: Image conversion is set to resze to", (IMG_SIZE, IMG_SIZE), "from", ORIGINAL_SIZE)

In [ ]:
from torchvision.ops import box_convert

# Create transforms
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Adjust brightness/contrast
    transforms.ToTensor(),
    transforms.Normalize(**IMG_NORM)
])

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(**IMG_NORM)
])

def target_transform(targets):
    """GOT10k bbox conversion from xywh to cxcywh format."""
    targets = [targets[0] / ORIGINAL_SIZE[0], targets[1] / ORIGINAL_SIZE[1], targets[2] / ORIGINAL_SIZE[0], targets[3] / ORIGINAL_SIZE[1]]
    tensors = torch.tensor(targets)
    return box_convert(tensors, 'xywh', 'cxcywh')

In [ ]:
# Create paired datasets with lazy loading
train_pairset = PairedGOT10kDataset(
    base_dataset=train_dataset, transform=train_transform, target_transform=target_transform
)
valid_pairset = train_pairset.extract_valid()
test_pairset = PairedGOT10kDataset(
    base_dataset=valid_dataset, transform=test_transform, target_transform=target_transform
)

print(f"INFO: PairedDataset initialized. Total sequences - Train({len(train_pairset)}), Valid({len(valid_pairset)}), Test({len(test_pairset)})")

In [ ]:
train_pairset[0]

In [ ]:
from torchvision.ops import box_convert

def visualize_frame_pair(pairset, idx=None, figsize=(7, 5)):
    """
    Visualize a pair of consecutive frames with their bounding boxes. (cxcy -> xyxy)

    Args:
        pairset: PairedGOT10kDataset instance
        idx: Index of the pair to visualize. If None, picks a random index
        figsize: Size of the figure as (width, height)
    """
    # Get random index if not provided
    if idx is None:
        idx = np.random.randint(len(pairset))

    # Get frame pair
    prev_img, curr_img, prev_gt, curr_gt = pairset[idx]

    # Convert tensors to numpy arrays and denormalize
    def denormalize(img_tensor):
        # Move channels to last dimension
        img = img_tensor.permute(1, 2, 0).numpy()
        # Denormalize
        img = img * np.array(IMG_NORM['std']) + np.array(IMG_NORM['mean'])
        # Clip values to valid range
        img = np.clip(img, 0, 1)
        return img

    prev_img = denormalize(prev_img)
    curr_img = denormalize(curr_img)

    def draw_bbox(ax, bbox, color='red'):
        """Helper function to draw bounding box"""
        x1, y1, x2, y2 = (_*IMG_SIZE for _ in box_convert(bbox, 'cxcywh', 'xyxy'))
        ax.plot([x1, x2], [y1, y1], color=color, linewidth=2)
        ax.plot([x1, x1], [y1, y2], color=color, linewidth=2)
        ax.plot([x2, x2], [y1, y2], color=color, linewidth=2)
        ax.plot([x1, x2], [y2, y2], color=color, linewidth=2)

    # Create figure
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)

    # Plot previous frame
    ax1.imshow(prev_img)
    draw_bbox(ax1, prev_gt)
    ax1.set_title('Previous Frame')
    ax1.axis('off')

    # Plot current frame
    ax2.imshow(curr_img)
    draw_bbox(ax2, curr_gt)
    ax2.set_title('Current Frame')
    ax2.axis('off')

    plt.tight_layout()
    plt.show()

    return idx

In [ ]:
for _ in range(2):
    selected_idx = visualize_frame_pair(train_pairset, _)
    print(f"Visualized pair index: {selected_idx}")

## DataLoader

In [ ]:
# Set Batch Size
BATCH_SIZE = 512, 512, 1

In [ ]:
# Use Teacher Forcing
train_pairset.use_teacher_forcing = True
valid_pairset.use_teacher_forcing = True

In [ ]:
MULTI_PROCESSING = True  # Set False if DataLoader is causing issues

from platform import system
if MULTI_PROCESSING and system() != "Windows":  # Multiprocess data loading is not supported on Windows
    import multiprocessing
    cpu_cores = multiprocessing.cpu_count()
    print(f"INFO: Number of CPU cores - {cpu_cores}")
else:
    cpu_cores = 0
    print("INFO: Using DataLoader without multi-processing.")

train_loader = DataLoader(train_pairset, batch_size=BATCH_SIZE[0], shuffle=True, num_workers=cpu_cores)
valid_loader = DataLoader(valid_pairset, batch_size=BATCH_SIZE[1], shuffle=False, num_workers=cpu_cores)
test_loader = DataLoader(test_pairset, batch_size=BATCH_SIZE[2], shuffle=False, num_workers=cpu_cores)

## Define Model
### APT: Adaptive Plugin for TTA (Test-time Adaptation)

In [ ]:
from transformers import RTDetrForObjectDetection, RTDetrImageProcessorFast, RTDetrConfig
from transformers.image_utils import AnnotationFormat
from safetensors.torch import load_file

In [ ]:
reference_model_id = "PekingU/rtdetr_r50vd"

# Load the reference model configuration
reference_config = RTDetrConfig.from_pretrained(reference_model_id, torch_dtype=torch.float32, return_dict=True)
reference_config.num_labels = 6

# Set the image size and preprocessor size
reference_config.image_size = 800

# Load the reference model image processor
reference_preprocessor = RTDetrImageProcessorFast.from_pretrained(reference_model_id)
reference_preprocessor.format = AnnotationFormat.COCO_DETECTION  # COCO Format / Detection BBOX Format
reference_preprocessor.size = {"height": IMG_SIZE, "width": IMG_SIZE}
reference_preprocessor.do_resize = False

In [ ]:
model_pretrained = RTDetrForObjectDetection(config=reference_config)
model_states = load_file("RT-DETR_R50vd_SHIFT_CLEAR.safetensors", device="cpu")
model_pretrained.load_state_dict(model_states, strict=False)

for param in model_pretrained.parameters():
    param.requires_grad = False  # Freeze

model_pretrained

In [ ]:
class FeatureNormalizationLayer(nn.Module):
    def __init__(self, target_dim=256):
        super().__init__()
        self.target_dim = target_dim

        # Keep only channel dimension
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))

        # Linear compression
        self.linear_compress = nn.AdaptiveAvgPool1d(self.target_dim)

        # Feature normalization
        self.feature_norm = nn.Sequential(
            nn.LayerNorm(target_dim),
            nn.Dropout(0.1)
        )

    def forward(self, x):
        # Apply adaptive pooling
        x = self.adaptive_pool(x)

        # Squeeze channel dimension
        x = x.squeeze(-1).squeeze(-1)

        # Linear compression
        x = self.linear_compress(x)

        # Feature normalization
        x = self.feature_norm(x)

        return x

In [ ]:
class APT(nn.Module):
    """
    Light-weight Sparse Autoencoder for Adaptation
    which learns how to sniff out the frame changes to predict next bounding boxes.
    """
    def __init__(self, feature_dim=256, bbox_dim=4, hidden_dim=32, sparsity_param=0.1):
        super().__init__()

        self.feature_dim = feature_dim
        self.bbox_dim = bbox_dim
        self.hidden_dim = hidden_dim
        self.sparsity_param = sparsity_param

        # Feature normalization layer for encoder-agnostic adaptation
        self.feature_norm = FeatureNormalizationLayer(target_dim=feature_dim)

        # Lightweight feature sniffer
        self.feature_sniffer = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 4 * 3)
        )

        # Previous bbox encoder
        self.bbox_encoder = nn.Sequential(
            nn.Linear(bbox_dim, hidden_dim // 4),
            nn.ReLU()
        )

        # Fusion layer
        self.fusion = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )

        # Prediction head
        self.predictor = nn.Sequential(
            nn.Linear(hidden_dim, bbox_dim),
            nn.Sigmoid()  # Normalize bbox coordinates to [0,1]
        )

        # Optional: Sparsity regularization
        self.activation = {}

    def forward(self, features, prev_bbox):
        # Normalize encoder features to be encoder-agnostic
        norm_features = self.feature_norm(features)

        # Extract relevant features from current frame
        sniffed_features = self.feature_sniffer(norm_features)

        # Encode previous bbox information
        bbox_features = self.bbox_encoder(prev_bbox)

        # Fuse features
        fused = self.fusion(
            torch.cat([sniffed_features, bbox_features], dim=-1)
        )

        # Predict next bbox
        next_bbox = self.predictor(fused)

        # Store activation for sparsity regularization if needed
        self.activation['hidden'] = fused

        return next_bbox

    def get_sparsity_loss(self):
        """Calculate sparsity regularization loss"""
        if 'hidden' not in self.activation:
            return 0

        rho_hat = torch.mean(self.activation['hidden'], dim=0)
        rho = torch.full_like(rho_hat, self.sparsity_param)

        # KL divergence for sparsity regularization
        sparsity_loss = torch.sum(
            rho * torch.log(rho/rho_hat) +
            (1-rho) * torch.log((1-rho)/(1-rho_hat))
        )

        return sparsity_loss

In [ ]:
class TestTimeAdaptiveDETR(nn.Module):
    def __init__(
        self, ?
    ):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.apt = APT(
            feature_dim=feature_dim, bbox_dim=bbox_dim,
            hidden_dim=hidden_dim, sparsity_param=sparsity_param
        )

    def forward(self, current_frame, prev_bbox, use_teacher_forcing=False):
        # Extract features using encoder
        features = self.encoder(current_frame)

        # Adapt using APT
        pred_bbox = self.apt(features, prev_bbox)

        if use_teacher_forcing:
            output = None
        else:
            # Use decoder for final prediction
            output = self.decoder(features)

        return output, pred_bbox, self.apt.get_sparsity_loss()

In [ ]:
# Initialize Model
model = TestTimeAdaptiveDETR()